In [4]:
from typing import List
import requests
import json
import ollama
import logging
import pandas as pd
import re


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class LlamaQASystem:
    def __init__(self, model_name: str = "llama3.2-vision"):
        self.model_name = model_name

    def get_prompt_by_type(self, query: str, question_type: str) -> str:
        base_system = (
            "您是一名專業獸醫，擅長：1.犬認知功能障礙綜合症（CCD）的診斷和護理 2.動物中醫相關問題 3.常見問題診斷及改善建議"
        )

        prompts = {
            "multiple_choice": f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
                                    {base_system}
                                    <|eot_id|><|start_header_id|>user<|end_header_id|>
                                    這是一題選擇題：{query}
                                    請以以下格式回答：
                                    1. 第一行只能回答 A/B/C/D (請勿帶任何標點、文字)
                                    2. 第二行才是說明，以 2~3 句話簡述理由
                                    3. 若同時出現多個選項，請只選一個最適合的
                                    4. 請在答案最後顯示你參考的來源連結或論文名稱，
                                    如果來源中包含「(經驗) some_link」，請在回答中以 [Experience: some_link] 形式標示；
                                    若包含「(文獻) some.pdf」，就 [reference: some.pdf]。
                                    5.  若有相關圖片，在回答中自然地說明圖片內容跟回答的關係

                                    限制：
                                    - 請將整體回答限制在 200 字以內
                                    - 直接切入重點
                                    - 保持客觀和專業
                                    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
                                    """,
            "true_false": f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
                                {base_system}
                                <|eot_id|><|start_header_id|>user<|end_header_id|>
                                問題：{query}
                                請依以下格式回答：
                                1. 一定先指出「True」或「False」(僅能擇一)，若遇到無法確定或證據不足的情況，一律為False，並說明原因
                                2. 再以 2～3 句話說明原因
                                3. 請在答案最後顯示你參考的來源連結或論文名稱，
                                    如果來源中包含「(經驗) some_link」，請在回答中以 [Experience: some_link] 形式標示；
                                    若包含「(文獻) some.pdf」，就 [reference: some.pdf]。
                                4.  若有相關圖片，在回答中自然地說明圖片內容跟回答的關係

                                限制：
                                - 請將整體回答限制在 200 字以內
                                - 直接切入重點
                                - 保持客觀和專業
                                <|eot_id|><|start_header_id|>assistant<|end_header_id|>
                                """,
            "qa": f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
                                {base_system}
                                <|eot_id|><|start_header_id|>user<|end_header_id|>
                                問題：{query}
                                請依以下格式回答：
                                1. 針對問題提供具體答案
                                2. 若遇到無法確定或證據不足的情況可以補充說明研究不足
                                3. 提供實用的建議或解釋
                                4. 請在答案最後顯示你參考的來源連結或論文名稱，
                                    如果來源中包含「(經驗) some_link」，請在回答中以 [Experience: some_link] 形式標示；
                                    若包含「(文獻) some.pdf」，就 [reference: some.pdf]。
                                5. 若有相關圖片，在回答中說明圖片內容

                                限制：
                                - 請將整體回答限制在 400 字以內
                                - 使用平易近人的語言
                                - 避免過度技術性術語
                                <|eot_id|><|start_header_id|>assistant<|end_header_id|>
                                """
        }
        return prompts.get(question_type, prompts["qa"])

    def determine_question_type(self, query: str) -> str:
        if any(char in query for char in ['A.', 'B.', 'C.', 'D.']) or any(keyword in query for keyword in ['何者', '下列', '哪一個', '哪種']):
            return "multiple_choice"
        elif any(keyword in query for keyword in ['是否', '是嗎', '對嗎', '正確嗎']):
            return "true_false"
        else:
            return "qa"

    def display_response(self, query: str):
            """Display response with text and images"""
            try:

                response_text = self.generate_response(query)
                
                print("Question:", query)
                print("\nSystem Response:")
                print(response_text)
                print("\n" + "-"*50 + "\n")

                return response_text 
                    
            except Exception as e:
                logger.error(f"Error in display_response: {str(e)}", exc_info=True)  
                return "", [] 

    def generate_response(self, query: str) -> str:
        question_type = self.determine_question_type(query)
        prompt = self.get_prompt_by_type(query, question_type)
        message = {
            'role': 'user',
            'content': prompt
        }
        response = ollama.chat(
            model=self.model_name,
            messages=[message]
        )
        return response['message']['content']

In [5]:
## single question test
# 測試查詢
qa_system = LlamaQASystem()

test_queries = [

# "針對年長犬隻可能出現的神經病理變化，哪些關鍵指標常被用來對比阿茲海默類型的退化症狀，並且與臨床觀察到的行為衰退有何關聯？",
# "除了藥物介入之外，平時飼養管理與環境調整方面有哪些具體作法，能同時有助於失智犬與失智貓維持較佳的生活品質，並為何多種方式並用的照護策略往往更能延緩認知退化？",
# "若以老犬作為模擬人類老化與失智的實驗模型，進行認知增益或治療性藥物的評估時，最常採用哪些評量方法來確認藥物對行為和神經功能的影響，並且在哪些神經傳導路徑上通常會看到較明顯的指標性變化？"
"六味地黃丸（Rehmannia 6）是臨床上治療犬貓慢性腎臟病屬「腎陰虛」證的常用基礎方，對嗎？",
"下列哪一味中藥最常用於治療犬隻熱性／濕性皮膚炎，具「清熱燥濕、解毒」之效？A) 黃芩（Scutellaria baicalensis）(B) 川芎(C) 生薑 (D) 陳皮",
"「獨活寄生湯」（又稱杜仲寄生湯）常用於老年犬退化性關節炎，屬風寒濕痹兼腎虛之證，對嗎？",
"針灸穴位「大椎」（GV14）位於第七頸椎與第一胸椎之間，臨床常用於退熱。下列何者為該穴之中文名稱？(A) 百會 (B) 大椎 (C) 三陰交 (D) 合谷",
"含麻黃（Ephedra）的處方因具升壓與興奮作用，對患有高血壓或心臟病的犬貓應避免使用，對嗎？",
"寒證過盛發生於陰氣過高而陽氣正常，對嗎？",
"用於免疫調節、風熱皮膚癢及降熱的要穴 LI‑11 曲池，其入針乾針深度約？(A) 0.3 寸 (B) 0.5 寸 (C) 1 寸( D) 1.5 寸",
"SI‑3 後谿的臨床主要適應症不包括下列何者？(A) 頸椎痛 (B) 腰椎間盤突出 (C) 耳炎 (D) 咳嗽",
"下列何者為安神、治療焦慮與失眠的常用心經穴，位置在腕橫紋尺側？(A) HT‑1 極泉( )B) HT‑3 少海 (C) HT‑7 神門 (D) HT‑9 少衝",
"LU‑7 列缺可用於犬頸椎痛、椎間盤疾病與面神經麻痺的針灸治療,對嗎？",
"位於前肢第 2、3 掌骨之間，常用於鼻塞與全身疼痛的腧穴是？(A) LI‑4 合谷 (B) LI‑10 前理 (C) LU‑7 列缺 (D) SI‑3 後谿",
"關於體液（Body Fluid）下列何者不正確？(A) 係全身正常液體總稱 (B) 滋潤營養身體 (C) 水腫、痰屬體液失調 (D) 乾燥毛皮是體液「過盛」"
                    ]

for query in test_queries:
    qa_system.display_response(query)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: 六味地黃丸（Rehmannia 6）是臨床上治療犬貓慢性腎臟病屬「腎陰虛」證的常用基礎方，對嗎？

System Response:
False

六味地黃丸（Rehmannia 6）雖然是臨床上常用的中藥，但它並不是治療犬貓慢性腎臟病的首選藥物。慢性腎臟病的治療需要根據個體情況進行調整，而六味地黃丸對於「腎陰虛」證的適用取決於具體的情況。

在臨床實踐中，我們需要綜合考慮犬貓的症狀、檢查結果和身體狀態，才能決定是否使用六味地黃丸。 [Reference: 中藥學習網]

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: 下列哪一味中藥最常用於治療犬隻熱性／濕性皮膚炎，具「清熱燥濕、解毒」之效？A) 黃芩（Scutellaria baicalensis）(B) 川芎(C) 生薑 (D) 陳皮

System Response:
B)

川芎具有清熱燥濕、解毒的功效，常用於治療犬隻熱性／濕性皮膚炎。它能夠清除體內的濕熱，消除瘙癢症狀，從而改善犬隻的舒適度。

[reference: 醫學書籍 "中藥物理論與實踐"]

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


...

Question: LU‑7 列缺可用於犬頸椎痛、椎間盤疾病與面神經麻痺的針灸治療,對嗎？

System Response:
False

LU-7 是一條重要的針灸穴位，但其主要用於治療呼吸系統問題，如喘息、咳嗽等，而非犬頸椎痛、椎間盤疾病或面神經麻痺。

在犬科針灸中，通常會使用不同的穴位來治療頸椎痛和椎間盤疾病，例如 GB-21 (風池)、LI-10 (之肢) 和 BL-10 (膈關)。這些穴位可以幫助減輕疼痛和改善神經功能。

有關面神經麻痺的治療，通常會使用 GV-20 (百會) 和 Zanz_3 (太陽交) 等穴位來調節神經系統和身體平衡。

參考來源：[reference: "Canine Acupuncture" by World Small Animal Veterinary Association]

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: 位於前肢第 2、3 掌骨之間，常用於鼻塞與全身疼痛的腧穴是？(A) LI‑4 合谷 (B) LI‑10 前理 (C) LU‑7 列缺 (D) SI‑3 後谿

System Response:
根據您的問題，答案是 (B) LI-10 前三里。

LI-10 是位於前肢第 2、3 掌骨之間的腧穴，常用於鼻塞和全身疼痛的治療。它是十二經脈中的任脈的一個重要點，也被稱為「前理」或「三里」。根據中醫理論，這個腧穴可以幫助調節氣機，釋放阻塞的氣血，並且有助於減少全身疼痛。

在臨床上，LI-10 膏藥灸或艾条灸都可用於刺激該腧穴。對於鼻塞，可以進行鼻內注射或使用藥物針劑來刺激該腧穴。由於這個腧穴也位於關節區域，因此有助於調理前肢的關節疼痛。

在實踐中，醫師應根據患者的症狀選擇合適的治療方法，並且注意安全性和個人化的因素。

參考來源：《內經》

圖片內容：無

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: 關於體液（Body Fluid）下列何者不正確？(A) 係全身正常液體總稱 (B) 滋潤營養身體 (C) 水腫、痰屬體液失調 (D) 乾燥毛皮是體液「過盛」

System Response:
A

體液是指人類或動物身體中的一種流體，包括血液、組織液、淋巴液等。它們的功能有滋潤、運輸、調節溫度等。選項 (D) 說的是乾燥毛皮是體液「過盛」，這是不正確的。乾燥毛皮通常是指皮膚缺乏足夠的濕潤，與體液過盛無關。

*   參考文獻：[體液的概念和功能](https://www.mayoclinic.org/zh-tw/diseases-conditions/hydrations/symptoms-causes/syc-20375195)

--------------------------------------------------



In [ ]:
def parse_llm_answer(llm_response: str, q_type: str) -> str:
    """
    根據題型 (選擇 or 是非)，從 LLM 的回覆字串中解析出可能的最終答案。
    """

    # 把回覆都轉小寫，以便搜尋
    text_lower = llm_response.lower().strip()
    cleaned = llm_response.strip()
    
    # 定義對應字典
    positive_responses = {'yes', '是', '對', 'true','True','Yes'}
    negative_responses = {'no', '否', '不對', 'false','False','No'}
    
    if q_type == "選擇":
       
        # 1) 先去除可能的全形/半形混雜、移除多餘符號等（可選）
        #    下面先做個最基本的 strip() 處理
        cleaned = llm_response.strip()

        # 2) 建立 Regex：
        #    - `^[ \t]*(A|B|C|D)[ \t]*$`：代表這一行(含前後空白)只有 A/B/C/D
        #    - (?m) 代表 MULTILINE 模式，使 ^ 和 $ 可以匹配每一行的開頭與結尾
        pattern = re.compile(r'^[ \t]*(A|B|C|D)[ \t]*$', re.MULTILINE)

        # 3) 搜尋
        match = pattern.search(cleaned)
        if match:
            # group(1) 會是 'A' or 'B' or 'C' or 'D'
            return match.group(1)
        else:
            return "UNKNOWN"
    
    elif q_type == "是非":
        # 將輸入的回應斷詞，檢查是否包含正向或負向的關鍵詞
        words = re.findall(r'\w+', text_lower)  # 取出所有單字
        
        # 如果找到任何正面回應
        if any(word in positive_responses for word in words):
            return "True"
        
        # 如果找到任何負面回應
        if any(word in negative_responses for word in words):
            return "False"
        
        return "UNKNOWN"
    
    else:
        return "UNKNOWN"
def main():
        df = pd.read_excel("test_questions.xlsx")
        
        test_df = df[df["type"].isin(["選擇", "是非"])].copy()
        
        test_df["llm_response"] = ""
        test_df["predicted"] = ""
        test_df["is_correct"] = 0

        qa_system = LlamaQASystem()
        
        for idx, row in test_df.iterrows():
            q = row["question"]
            q_type = row["type"]
            correct_ans = str(row["answers"]).strip()
            
            response_text = qa_system.display_response(q)
            pred_ans = parse_llm_answer(response_text, q_type)
            
            is_correct = 1 if pred_ans.upper() == correct_ans.upper() else 0
            
            test_df.at[idx, "llm_response"] = response_text
            test_df.at[idx, "predicted"] = pred_ans
            test_df.at[idx, "is_correct"] = is_correct
        
        total = len(test_df)
        correct_count = test_df["is_correct"].sum()
        accuracy = correct_count / total if total > 0 else 0.0
        
        print("=== 測試結果 ===")
        print(test_df[["id", "type", "answers", "predicted", "is_correct"]])
        print(f"\n共 {total} 題，正確 {correct_count} 題，Accuracy = {accuracy:.2f}")
        
        test_df.to_csv("test_result_baseline.csv", index=False, encoding='utf-8')
        print("結果已儲存 test_result_baseline.csv")
if __name__ == "__main__":
    # qa_system = LlamaQASystem()  # 創建 LlamaQASystem 的實例
    # qa_system.main("test_questions.xlsx")  # 呼叫實例的 main() 方法
    main()
    


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: CCD 是否與神經發炎相關？是否有特定細胞因子（cytokines）或發炎路徑（例如NLRP3 inflammasome）參與？

System Response:
True

CCD（犬認知功能障礙綜合症）是一種複雜的疾病，目前尚未完全了解其神經發炎機制。然而，一些研究表明發炎反應可能在疾病進展中發揮作用。例如，某些研究發現NLRP3 inflammasome活性增高與CCD相關 [reference: some.pdf]。

圖片：一張顯示犬隻神經組織中的發炎細胞的微鏡照片，強調了發炎在疾病進展中的作用。

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: CCD 是否與腸道微生物群變化有關？哪些特定細菌群可能影響大腦健康？

System Response:
True

犬認知功能障礙綜合症（CCD）的成因複雜，研究表明腸道微生物群變化可能是其中一個重要影響因素。研究指出某些特定的細菌群，如肠道益生菌，在保持腦健康方面發揮關鍵作用。例如，Bifidobacterium 和 Lactobacillus 等菌種能夠通過制備神經遞質、調節免疫反應等方式影響大腦功能。

圖片: 猫的脑与肠道微生物群的交互作用示意图（注意：此图仅供参考）

[參考] Bercik P, et al. (2012). The microbiota and the regulation of the immune system. Journal of Clinical Investigation, 122(1), 22-30.

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


...

Question: 文獻中提到犬隻認知功能障礙 (CCD) 與中樞神經免疫反應的關聯，下列哪種細胞或炎症小體常被視作潛在研究重點？
A. Osteoclast 骨細胞
B. Microglia 與 NLRP3 inflammasome
C. 肌肉纖維與 CK (Creatine Kinase)
D. 紅血球大小 (MCV)

System Response:
B

文獻中提到犬隻認知功能障礙 (CCD) 與中樞神經免疫反應的關聯，研究者們正在探索中樞神經系統中炎症機制與 CCD 之間的關聯。Microglia 是中樞神經系統的第一型免疫細胞，其活化和功能異常可能是 CCD 的潛在原因之一。NLRP3 inflammasome是一種複合體，參與了發生炎症反應的調節。它們的活化會導致炎症介質的釋放，從而影響中樞神經系統的功能。

[reference: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6451918/]

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: 犬隻服用人類認知增強藥物（如前驅膽鹼類、抗氧化劑等）已被證實能完全逆轉老犬的 CCD 症狀，正確嗎。

System Response:
False

目前尚未有任何科學證據表明犬隻服用人類認知增強藥物能夠完全逆轉老犬的 CCD 症狀。這些藥物可能對於某些個體提供短期改善，但長期來說效果並不穩定，且可能出現副作用。

犬隻的 CCD是一種複雜的疾病，需要綜合治療和管理。目前主要的治療方法包括行為治療、藥物治療（如塞連他嗪和阿米替林）和改善生活環境等。

參考:[reference: Canine Cognitive Dysfunction Syndrome.pdf]

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: 針對犬隻失智 (CCD) 的診斷，是否可以單靠 MRI 或 CT 等影像學觀察大腦萎縮或斑塊沉積，即可準確區分 CCD 與其他神經退化疾病。

System Response:
True

單靠 MRI 或 CT 等影像學觀察大腦萎縮或斑塊沉積不足以準確區分犬隻失智 (CCD) 與其他神經退化疾病。因為許多神經退化疾病都會導致類似的腦部結構變化，例如腦垂腺瘤、腦血管疾病等。

影像學檢查可以提供一些線索，但需要結合臨床表現和實驗室檢查結果才能進行確診。例如，CCD 的患者可能會出現記憶力和認知功能的下降，而這些症狀與其他神經退化疾病可能有所不同。

因此，影像學檢查應該與臨床評估結合使用，以取得更準確的診斷結果。[reference: Canine Cognitive Dysfunction Syndrome]

圖片：一張 MRI掃描結果，顯示大腦萎縮區域（紅色標記），但這種變化可能出現在多種神經退化疾病中，需要結合臨床表現進行診斷。

--------------------------------------------------

=== 測試結果 ===
    id type answers predicted  is_correct
5    6   是非    True      True           1
6    7   是非    True      True           1
7    8   是非    True     False           0
8    9   是非    True     False           0
11  12   是非   False      True           0
12  13   是非   False      True           0
13  14   是非   False      True           0
15  16   是非    True     False           0
16  17   是非    True      True           1
22  24   選擇       C         C           1
23  25   選擇       B         B        